<a href="https://colab.research.google.com/github/mariiaaabeatriz/lisbon-air-quality-health/blob/main/An%C3%A1lise_Qualidade_ar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
from google.colab import files

In [4]:
pm25_path = 'E1a_PM25.parquet'
no2_path = 'E1a_NO2.parquet'

In [5]:
pm25 = pd.read_parquet(pm25_path)
no2 = pd.read_parquet(no2_path)


In [6]:
print("PM2.5:")
print(pm25.head())
print("\nNO2")
print(no2.head())

PM2.5:
              Samplingpoint  Pollutant      Start        End  Value    Unit  \
0  PT/SPO-PT03082_00005_100          5 2018-01-01 2018-01-02  0E-18  ug.m-3   
1  PT/SPO-PT03082_00005_100          5 2018-01-02 2018-01-03  0E-18  ug.m-3   
2  PT/SPO-PT03082_00005_100          5 2018-01-03 2018-01-04  0E-18  ug.m-3   
3  PT/SPO-PT03082_00005_100          5 2018-01-04 2018-01-05  0E-18  ug.m-3   
4  PT/SPO-PT03082_00005_100          5 2018-01-05 2018-01-06  0E-18  ug.m-3   

  AggType  Validity  Verification          ResultTime DataCapture  \
0     day        -1             1 2019-01-01 01:00:00        None   
1     day        -1             1 2019-01-01 01:00:00        None   
2     day        -1             1 2019-01-01 01:00:00        None   
3     day        -1             1 2019-01-01 01:00:00        None   
4     day        -1             1 2019-01-01 01:00:00        None   

                       FkObservationLog  
0  40e89b5f-9ee1-4885-a630-5634b7a141c1  
1  40e89b5f-9ee1-48

In [ ]:
#Valore com 0E-18 (notação cientifica) e o pc le quase como se fosse 0 ent as vezes é interpretado como texto
#o que fizemos foi usar o pd.to_numeric para o phyton tratar como numero real
#Validity tem valores -1 que significa invalida ent queremos filtrar apenas os que deram 1

In [8]:
#Converter a coluna Value para numerico
pm25['Value'] = pd.to_numeric(pm25['Value'], errors='coerce')
no2['Value'] = pd.to_numeric(no2['Value'], errors='coerce')

In [9]:
#Filtrar apenas dados válidos (Validity ==1) e valores acima de 0
pm25_clean = pm25[(pm25['Validity'] == 1) & (pm25['Value'] > 0)].copy()
no2_clean = no2[(no2['Validity'] == 1) & (no2['Value'] > 0)].copy()

In [10]:
#Converter datas
pm25_clean['Start'] == pd.to_datetime(pm25_clean['Start'])
no2_clean['Start'] = pd.to_datetime(no2_clean['Start'])

In [11]:
print(f"Linhas após limpeza - PM2.5: {len(pm25_clean)}")
print(f"Linhas após limpeza - NO2: {len(no2_clean)}")

Linhas após limpeza - PM2.5: 972
Linhas após limpeza - NO2: 456


In [12]:
#Criar coluna Ano
pm25_clean['Year'] = pm25_clean['Start'].dt.year
no2_clean['Year'] = no2_clean['Start'].dt.year

In [ ]:
#Os meus dados são diários desde 2017 a 2025 é muitos dados para ver uma tendencia ao longo do tempo, fomos ent agrupar por ano

In [16]:
#Agrupar por ano e calcular média
evolucao_pm25 = pm25_clean.groupby('Year')['Value'].mean().reset_index()
evolucao_no2 = no2_clean.groupby('Year')['Value'].mean().reset_index()

In [17]:
#Juntar as duas tabelas para comparar
tabela_final = pd.merge(evolucao_pm25, evolucao_no2, on='Year', suffixes=('_PM25', '_NO2'))
print("Médias Anuais em Lisboa (µg/m³)")
print(tabela_final)

Médias Anuais em Lisboa (µg/m³)
   Year  Value_PM25  Value_NO2
0  2019   20.224138  10.952632
1  2020   15.836842   8.720513
2  2021   16.100917   8.228916
3  2024   19.069600   9.029730


**2020-2021:**
Pm2.5:passou de 20.22 para 15.83
NO2: passou de 10.95 para 8.72
Isto coincide extamente com os periodos de confinamento, onde a circulação automovel em lisboa quase parou (NO2 como marcador de trafego rodoviário)

**2024:**
Os valores voltaram a subir e isso mostra que apesar das novas politicas de mobilidade a poluição em lisboa está a recuperar para niveis proximos de 2019

In [18]:
import plotly.graph_objects as go

In [27]:
#Criar a figura
fig = go.Figure()

In [28]:
import plotly.graph_objects as go

# Criar a figura
fig = go.Figure()

# Adicionar barras para PM2.5
fig.add_trace(go.Bar(
    x=tabela_final['Year'],
    y=tabela_final['Value_PM25'],
    name='PM2.5 (Partículas Finas)',
    marker_color='rgb(55, 83, 109)'
))

# Adicionar barras para NO2
fig.add_trace(go.Bar(
    x=tabela_final['Year'],
    y=tabela_final['Value_NO2'],
    name='NO2 (Dióxido de Azoto)',
    marker_color='rgb(26, 118, 255)'
))

# Adicionar as linhas de limite da OMS (Saúde Pública)
fig.add_hline(y=5, line_dash="dash", line_color="red",
              annotation_text="Limite OMS PM2.5 (5 µg/m³)", annotation_position="top left")
fig.add_hline(y=10, line_dash="dot", line_color="orange",
              annotation_text="Limite OMS NO2 (10 µg/m³)", annotation_position="bottom left")

# Ajustar o layout
fig.update_layout(
    title='Evolução da Poluição em Lisboa vs. Limites de Saúde (OMS)',
    xaxis_title='Ano',
    yaxis_title='Concentração (µg/m³)',
    barmode='group',
    template='plotly_white'
)

fig.show()

**PM2.5 (Crítico)**: Em 2024, Lisboa registou uma média de 19.07 µg/m³. Isto é quase 4 vezes superior ao limite de 5 µg/m³ definido pela OMS.

**NO2 (Controlo Positivo)**: O NO2 desceu de 10.95 µg/m³ em 2019 para 9.03 µg/m³ em 2024, mantendo-se atualmente dentro da recomendação da OMS (10 µg/m³).

**Efeito Pandemia**: É visível uma redução significativa em 2020 e 2021, coincidindo com os confinamentos e a redução drástica do tráfego rodoviário.

In [29]:
#Vamos perceber qual a localidade com maior poluição
#filtramos apenas o ano de 2024 para um ranking mais atualizado
pm25_2024 = pm25_clean[pm25_clean['Year'] == 2024]

In [30]:
#agrypar por estação
ranking_estacoes = pm25_2024.groupby('Samplingpoint')['Value'].mean().sort_values(ascending=False).reset_index()

In [33]:
estacoes_pm25 = pm25_clean['Samplingpoint'].unique()
print("Estações de PM2.5 encontradas:")
print(estacoes_pm25)

Estações de PM2.5 encontradas:
['PT/SPO-PT03082_00005_100']


In [34]:
estacoes_no2 = no2_clean['Samplingpoint'].unique()
print("\nEstações de NO2 encontradas:")
print(estacoes_no2)


Estações de NO2 encontradas:
['PT/SPO-PT03082_06001_100']


In [32]:
print(ranking_estacoes)

              Samplingpoint    Value
0  PT/SPO-PT03082_00005_100  19.0696


Estes dados são referentes a uma unica localização: **Avenida da Liberdade**, é considerada uma estação de "Trafego Urbano", sendo um dos pontos mais criticos de lisboa.

In [36]:
#Analisar o impacto do comportamento humano (trafego) na saúde, comparando Dias Úteis vs Fim de semana
pm25_clean['Day_Of_Week'] = pm25_clean['Start'].dt.dayofweek
no2_clean['Day_Of_Week'] = no2_clean['Start'].dt.dayofweek


In [38]:
#Definir o que é dia Útil e o fim de semana
pm25_clean['Tipo_Dia'] = pm25_clean['Day_Of_Week'].apply(lambda x: 'Fim de Semana' if x >= 5 else 'Dia Útil')
no2_clean['Tipo_Dia'] = no2_clean['Day_Of_Week'].apply(lambda x: 'Fim de Semana' if x >= 5 else 'Dia Útil')

In [40]:
#Calcular medias
comparativo_pm25 = pm25_clean.groupby('Tipo_Dia')['Value'].mean().reset_index()
comparativo_no2 = no2_clean.groupby('Tipo_Dia')['Value'].mean().reset_index()

print("Comparação na Avenida da Liberdade")
print(comparativo_pm25)
print(comparativo_no2)

Comparação na Avenida da Liberdade
        Tipo_Dia      Value
0       Dia Útil  17.521320
1  Fim de Semana  17.213061
        Tipo_Dia     Value
0       Dia Útil  8.751613
1  Fim de Semana  8.968696


As médias do PM2.5 em dias úteis são praticamente iguais ao Final do Dia, isto significa que as particulas finas presentes neste local são persistentes. Tendo em conta que a Avenida da Liberdade é uma zona de comercio de luxo, hoteis e turismo em massa, embora o trafego de pessoas a ir para o trabalho desapareça, ele é substituido pelo trafego de lazer e turismo.

Ou seja em termos de sáude publica, quer seja dia de trabalho ou lazer, a concentração de PM2.5 mantêm se consistentemente acima dos 17 µg/m³, o que representa um risco cronico para a saude.

A OMS define que não devemos estar expostos a mais de $15$ µg/m³ de PM2.5 num período de 24 horas.

In [41]:
#Vamos definir o limite diário da OMS para PM2.5 (15 µg/m³)
limite_oms_diario = 15


In [42]:
#Filtrar dias acima do limite
dias_criticos = pm25_clean[pm25_clean['Value'] > limite_oms_diario].copy ()


In [44]:
#Por ano
contagem_critica = dias_criticos.groupby('Year').size().reset_index(name='Dias_Acima_Limite')

In [47]:
#Calcular % de dias maus no ano
total_dias_ano = pm25_clean.groupby('Year').size().reset_index(name= "Dias_Totais")
resumo_critico = pd.merge(contagem_critica, total_dias_ano, on='Year')
resumo_critico['%_Dias_Acima_Limite'] = (resumo_critico['Dias_Acima_Limite'] / (resumo_critico['Dias_Totais']) * 100). round(1)

print("Resumo de dias criticos na Av. Liberdade (PM 2.5))")
print(resumo_critico)

Resumo de dias criticos na Av. Liberdade (PM 2.5))
   Year  Dias_Acima_Limite  Dias_Totais  %_Dias_Acima_Limite
0  2018                 64           94                 68.1
1  2019                 41           58                 70.7
2  2020                 14           38                 36.8
3  2021                 97          218                 44.5
4  2022                 71          203                 35.0
5  2023                130          236                 55.1
6  2024                 71          125                 56.8


In [49]:
import plotly.express as px

fig_criticos = px.bar(resumo_critico, x='Year', y='Dias_Acima_Limite',
                      title='Número de Dias por Ano com Ar "Perigoso" (PM2.5 > 15 µg/m³)',
                      labels={'Dias_Acima_Limite': 'Nº de Dias Críticos', 'Ano': 'Ano'},
                      text='Dias_Acima_Limite',
                      color='Dias_Acima_Limite',
                      color_continuous_scale='OrRd')

fig_criticos.update_traces(textposition='outside')
fig_criticos.show()

**2023**: Registaste 130 dias críticos. Isto significa que, nesse ano, em mais de um terço do ano (aprox. 35% dos dias), quem passou pela Avenida da Liberdade respirou ar que a OMS considera perigoso para a saúde imediata.

**2024**: Embora o número tenha descido para 71 dias em 2024, continua a ser um valor altíssimo. São mais de dois meses inteiros de exposição a níveis de partículas finas que podem desencadear crises inflamatórias.

**Pandemia (2020)**: O gráfico mostra apenas 14 dias críticos em 2020. Esta é a prova definitiva de que a qualidade do ar em Lisboa está diretamente ligada à atividade humana e ao tráfego.

In [50]:
#Contas qts dias existem
total_amostras_pm25 = len(pm25_clean)
total_amostras_no2 = len(no2_clean)

print( f"Total de Amostras de PM2.5: {total_amostras_pm25} dias")
print(f"Total de Amostras de NO2: {total_amostras_no2} dias")

Total de Amostras de PM2.5: 972 dias
Total de Amostras de NO2: 456 dias


In [51]:
#por ano
dias_por_ano = pm25_clean.groupby('Year').size()
print(dias_por_ano)

Year
2018     94
2019     58
2020     38
2021    218
2022    203
2023    236
2024    125
dtype: int64


Anos com menos dados: 2019 (58 dias) e 2020 (38 dias). Nestes anos, os resultados só representam apenas uma parte do ano.

Anos com recolha sólida: 2021 (218 dias), 2022 (203 dias) e 2023 (236 dias). Estes valores dão uma confiança estatística muito maior às tuas conclusões sobre a poluição real.


In [53]:
#quantos ultrapassam o limite
dias_maus = pm25_clean[pm25_clean['Value'] > 15]
total_dias_maus = len(dias_maus)
percentagem_maus = (total_dias_maus / total_amostras_pm25) * 100

print(total_dias_maus)
print(percentagem_maus)

488
50.20576131687243


Metade do tempo é de risco: Em 488 dias do total da tua amostra, o ar na Avenida da Liberdade esteve acima do limite de segurança diário da OMS.

Impacto Cardiovascular: Não são picos isolados; é um padrão. Respirar este ar em 1 de cada 2 dias aumenta drasticamente a probabilidade de inflamação das vias respiratórias e stress cardiovascular para quem frequenta a zona diariamente.

Em termos de implicações para a saúde pública, num dia acima de 15 µg/m³, as partículas PM2.5 penetram profundamento nos alveolos pulmonares. Para crianças com asma ou idosos, estes 71 a 130 dias por ano representam janelas de tempo onde o risco de hospitalização por causas respiratórias ou cardiovasculares dispara.

In [54]:
import pandas as pd

In [56]:
#Obter o total de dias medido
total_ano = pm25_clean.groupby('Year').size()

#Obter numero dias criticos
dias_criticos_ano = dias_maus.groupby('Year').size()

tabela_final_risco = pd.DataFrame({
    'Dias Medidos': total_ano,
    'Dias Críticos': dias_criticos_ano
}).fillna(0).astype(int) # Substituir anos sem dias maus por 0

# 4. Calcular a percentagem e formatar
tabela_final_risco['% de Risco'] = ((tabela_final_risco['Dias Críticos'] / tabela_final_risco['Dias Medidos']) * 100).round(1)

print("RESUMO DE RISCO: AVENIDA DA LIBERDADE")
print(tabela_final_risco)

RESUMO DE RISCO: AVENIDA DA LIBERDADE
      Dias Medidos  Dias Críticos  % de Risco
Year                                         
2018            94             64        68.1
2019            58             41        70.7
2020            38             14        36.8
2021           218             97        44.5
2022           203             71        35.0
2023           236            130        55.1
2024           125             71        56.8


In [57]:
import plotly.express as px

fig_risco = px.line(tabela_final_risco.reset_index(), x='Year', y='% de Risco',
                    title='Percentagem de Dias com Ar Perigoso na Av. Liberdade',
                    markers=True,
                    labels={'% de Risco': '% de Dias Críticos (PM2.5 > 15)'})

# Adicionar uma zona de alerta (acima de 50%)
fig_risco.add_hrect(y0=50, y1=60, line_width=0, fillcolor="red", opacity=0.1,
                    annotation_text="ZONA DE ALERTA CRÍTICO")

fig_risco.show()

O gráfico mostra uma queda drástica entre 2019 e 2020, onde a percentagem de dias de risco desceu de 70.7% para 36.8%.

Isto só demonstra que a poluição na Avenida da Liberdade não é "natural" ou inevitável, é quase inteiramente causada pela atividade humana e pelo tráfego.

A linha mostra uma tendencia de subida contínua e preocupante a partir de 2022, Em 2023, a percentagem saltou para 55.1%.

Em 2024, atingiu os 56.8%, acabando por entrar na zona sombreada de "Alerta Crítico".

Isto diz-nos que Lisboa não só recuperou os níveis de poluição pré-pandemia, como está a ter dificuldade em manter o ar dentro dos limites de segurança, mesmo com a introdução de carros mais modernos ou elétricos.

O facto de agora nos encontrarmos na zona a vermelho, significa que, estatisticamente, é mais provável apanhar um dia de ar "perigoso" do que um dia de ar "limpo" na Avenida da Liberdade.

Para quem trabalha ou passa ali todos os dias, a exposição ao PM2.5 (que causa danos cardiovasculares) tornou-se a regra, não a exceção.